# Drive 연동

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# import os

# def count_images_in_directory(root_dir):
#     # 이미지 파일 확장자들
#     image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')
#     image_count = 0

#     # os.walk를 사용하여 모든 하위 폴더 탐색
#     for root, dirs, files in os.walk(root_dir):
#         for file in files:
#             # 이미지 확장자인지 확인
#             if file.lower().endswith(image_extensions):
#                 image_count += 1

#     return image_count

# # 경로 설정
# root_dir = '/content/drive/MyDrive/Data/yolo_crops'

# # 이미지 개수 출력
# print(f"Total number of images: {count_images_in_directory(root_dir)}")

Total number of images: 2875


# Roboflow를 통해 라벨링 해두었던 데이터셋 다운로드
* 데이터셋은 수동으로 라벨링함.
* Flip: Horizontal, Rotation: Between -15° and +15°으로 데이터 3배 증강

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aGuH7ZZSLKqF5erRVTuu")
project = rf.workspace("dfd-wcvot").project("cardd_repair_replace")
version = project.version(2)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


# yolo v5 모델 다운로드

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

%cd /content/yolov5/
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


확장자 확인

In [ ]:
import os

# Define the directory path
directory_path = "/content/CarDD_repair_replace-2/valid/images"

# Get the list of files in the directory
try:
    files = os.listdir(directory_path)

    # Extract the file extensions and remove duplicates
    extensions = set(os.path.splitext(file)[1] for file in files if os.path.isfile(os.path.join(directory_path, file)))

    # Output the unique extensions
    print(extensions)
except FileNotFoundError:
    print("The specified directory does not exist.")

{'.jpg'}


In [ ]:
from glob import glob

train_img_list = glob('/content/CarDD_repair_replace-2/train/images/*.jpg')
valid_img_list = glob('/content/CarDD_repair_replace-2/valid/images/*.jpg')
test_img_list = glob('/content/CarDD_repair_replace-2/test/images/*.jpg')

print(f"train data: {len(train_img_list)}개")
print(f"validation data: {len(valid_img_list)}개")
print(f"test data: {len(test_img_list)}개")

train data: 2685개
validation data: 256개
test data: 129개


# data.yaml을 우리 데이터 포맷에 맞춰 바꿈.

In [ ]:
with open('/content/CarDD_repair_replace-2/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/CarDD_repair_replace-2/val.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

with open('/content/CarDD_repair_replace-2/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

In [ ]:
import yaml

with open('/content/CarDD_repair_replace-2/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/CarDD_repair_replace-2/train.txt'
data['val'] = '/content/CarDD_repair_replace-2/val.txt'
data['test'] = '/content/CarDD_repair_replace-2/test.txt'

with open('/content/CarDD_repair_replace-2/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['repair', 'replace'], 'nc': 2, 'roboflow': {'license': 'CC BY 4.0', 'project': 'cardd_repair_replace', 'url': 'https://universe.roboflow.com/dfd-wcvot/cardd_repair_replace/dataset/2', 'version': 2, 'workspace': 'dfd-wcvot'}, 'test': '../test/images', 'train': 'CarDD_repair_replace-2/train/images', 'val': 'CarDD_repair_replace-2/valid/images'}
{'names': ['repair', 'replace'], 'nc': 2, 'roboflow': {'license': 'CC BY 4.0', 'project': 'cardd_repair_replace', 'url': 'https://universe.roboflow.com/dfd-wcvot/cardd_repair_replace/dataset/2', 'version': 2, 'workspace': 'dfd-wcvot'}, 'test': '/content/CarDD_repair_replace-2/test.txt', 'train': '/content/CarDD_repair_replace-2/train.txt', 'val': '/content/CarDD_repair_replace-2/val.txt'}


# Training

In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/CarDD_repair_replace-2/data.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt --name yolov5x_results

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
      35/49      7.58G     0.0298    0.01378  0.0006584         57        416:  38% 63/168 [00:35<01:03,  1.66it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      35/49      7.58G    0.02977    0.01372  0.0006511         38        416:  38% 64/168 [00:36<01:04,  1.60it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      35/49      7.58G    0.02978    0.01372  0.0006426         45        416:  39% 65/168 [00:36<01:01,  1.68it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      35/49      7.58G    0.02976    0.01367  0.0

# 훈련한 가중치 Drive에 복사

In [ ]:
!cp -r /content/yolov5/runs/train/yolov5x_results /content/drive/MyDrive/models_repair_replace_2

# 훈련한 가중치를 이용하여 모델이 예측한 라운딩 박스만 잘라내어 저장

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/models_repair_replace_2/yolov5x_results/weights/best.pt --img 320 --conf 0.5 --source /content/CarDD_repair_replace-2/test/images --save-crop

detect: weights=['/content/drive/MyDrive/models_repair_replace_2/yolov5x_results/weights/best.pt'], source=/content/CarDD_repair_replace-2/test/images, data=data/coco128.yaml, imgsz=[320, 320], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-363-gf44db010 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/129 /content/CarDD_repair_replace-2/test/images/repair1003_png.rf.148a3ea3a1164cfc2608a90f5406123f.jpg: 320x320 1 repair, 34.8ms
image 2/129 /content/CarDD_repair_replace-2/test/images/repai

In [ ]:
# import shutil
# import os

# # 삭제할 디렉토리 경로
# directory_path = "/content/yolov5/runs/detect/exp"

# # 디렉토리와 그 안의 모든 내용 삭제
# if os.path.exists(directory_path):
#     shutil.rmtree(directory_path)
#     print(f"Successfully deleted all data in {directory_path}.")
# else:
#     print(f"Directory {directory_path} does not exist.")

Successfully deleted all data in /content/yolov5/runs/detect/exp.


In [ ]:
import os
import shutil

# 원본 파일 경로
source_dir = '/content/yolov5/runs/detect/exp/crops/replace'

# Google Drive의 목표 경로
destination_dir = '/content/drive/MyDrive/Data/yolo_crops_repair_replace/test/replace'

# 디렉토리가 존재하지 않으면 생성
os.makedirs(destination_dir, exist_ok=True)

# 디렉토리 내의 모든 파일 복사
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    destination_file = os.path.join(destination_dir, filename)

    # 파일인지 확인 후 복사
    if os.path.isfile(source_file):
        shutil.copy(source_file, destination_file)

In [ ]:
# %cd /
# from glob import glob

# bulls = len(glob('/content/drive/MyDrive/Data/yolo_crops/BullsEye/*.jpg'))
# combi = len(glob('/content/drive/MyDrive/Data/yolo_crops/CombinationBreak/*.jpg'))
# crack = len(glob('/content/drive/MyDrive/Data/yolo_crops/Crack/*.jpg'))
# etc = len(glob('/content/drive/MyDrive/Data/yolo_crops/Etc/*.jpg'))
# star = len(glob('/content/drive/MyDrive/Data/yolo_crops/StarBreak/*.jpg'))
# stone = len(glob('/content/drive/MyDrive/Data/yolo_crops/StoneChip/*.jpg'))

# print(f"총 데이터셋 크기: {bulls + combi + crack + etc + star + stone}")

/
총 데이터셋 크기: 2885


# Evaluate

In [ ]:
# !python /content/yolov5/val.py --weights /content/drive/MyDrive/models/weights/best.pt --data /content/CarDD-3/data.yaml --img 320 --iou 0.65 --half

val: data=/content/CarDD-3/data.yaml, weights=['/content/drive/MyDrive/models/weights/best.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.65, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=content/yolov5/runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-345-g8003649c Python-3.10.12 torch-2.3.1+cu121 CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86207059 parameters, 0 gradients, 203.9 GFLOPs
100% 755k/755k [00:00<00:00, 27.8MB/s]
val: Scanning /content/CarDD-3/val... 238 images, 0 backgrounds, 0 corrupt: 100% 238/238 [00:00<00:00, 393.54it/s]
val: New cache created: /content/CarDD-3/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/8 [00:00<?, ?it/s]^C


# 실제 라벨에 맞는 곳으로 파일을 이동
* 실제 라벨은 BullsEye인데, yolo 모델이 CombinationBreak로 예측한 경우에 CombinationBreak 폴더에 저장이 되어 있음.
* 따라서, 각 폴더에서 자신의 라벨이 아닌 파일들은 제자리로 이동시킴.

In [ ]:
import os
import shutil

# 경로 설정
base_dir = '/content/drive/MyDrive/Data/yolo_crops_repair_replace/test'
repair_dir = os.path.join(base_dir, 'repair')
replace_dir = os.path.join(base_dir, 'replace')

# 파일 이동 함수
def move_files(source_dir, target_dir, keyword):
    files = os.listdir(source_dir)

    for file in files:
        if file.endswith(".jpg"):  # 이미지 파일만 처리
            if keyword in file:
                # 파일 이동
                shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, file))

# repair 폴더에서 replace 키워드가 있는 파일을 replace 폴더로 이동
move_files(repair_dir, replace_dir, 'replace')

# replace 폴더에서 repair 키워드가 있는 파일을 repair 폴더로 이동
move_files(replace_dir, repair_dir, 'repair')

print("파일 이동 작업이 완료되었습니다.")

파일 이동 작업이 완료되었습니다.


# 이동 전보다 10장이 줄음. 중복된 데이터가 있어서 이름이 같은 경우가 있을 수 있음. --> 중복 데이터 10장이 제거됨.

In [ ]:
%cd /
from glob import glob

bulls = len(glob('/content/drive/MyDrive/Data/yolo_crops_repair_replace/test/repair/*.jpg'))
combi = len(glob('/content/drive/MyDrive/Data/yolo_crops_repair_replace/test/replace/*.jpg'))
crack = len(glob('/content/drive/MyDrive/Data/yolo_crops_repair_replace/train/repair/*.jpg'))
etc = len(glob('/content/drive/MyDrive/Data/yolo_crops_repair_replace/train/replace/*.jpg'))
star = len(glob('/content/drive/MyDrive/Data/yolo_crops_repair_replace/valid/repair/*.jpg'))
stone = len(glob('/content/drive/MyDrive/Data/yolo_crops_repair_replace/valid/replace/*.jpg'))

print(f"총 데이터셋 크기: {bulls + combi + crack + etc + star + stone}")

/
총 데이터셋 크기: 4474


* 여기서 직접 train, test, valid 데이터를 생성할 수 있네.
* 어차피, roboflow에서 데이터셋을 나눌 때 증강한 데이터는 train에만 넣으니, 두 번 작업할 필요가 없었음.